In [5]:

import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In [6]:
from sklearn.pipeline import make_pipeline


In [7]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2026/02/02 16:31:06 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp-exercises/mlartifacts/2', creation_time=1770049866466, experiment_id='2', last_update_time=1770049866466, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [9]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [14]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [17]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

2026/02/02 16:59:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2026/02/02 16:59:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run rebellious-fowl-450 at: http://127.0.0.1:5000/#/experiments/2/runs/b5a057ab575540b19a24dfcb60e0a5f7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [18]:
from mlflow.tracking import MlflowClient


In [19]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'b5a057ab575540b19a24dfcb60e0a5f7'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [21]:
import mlflow
logged_model = f'runs:/{RUN_ID}/model'

# Load model as a PyFunc Model.
loaded_model = mlflow.pyfunc.load_model(logged_model)

print("Model loaded successfully!")

Model loaded successfully!


In [24]:
import os
import pickle

# 1. Κατεβάζουμε ξανά το artifact για να είμαστε σίγουροι ότι το 'path' ορίζεται
RUN_ID = 'b5a057ab575540b19a24dfcb60e0a5f7' # Βάλε το δικό σου ID εδώ
path = client.download_artifacts(run_id=RUN_ID, path='model')

# 2. Τώρα που το 'path' ορίστηκε, φτιάχνουμε το model_file_path
model_file_path = os.path.join(path, 'model.pkl')

# 3. Φορτώνουμε με pickle
with open(model_file_path, 'rb') as f_in:
    loaded_pipeline = pickle.load(f_in)

print("Τώρα το φορτώσαμε σωστά!")

Τώρα το φορτώσαμε σωστά!
